In [8]:
import pandas as pd
import numpy as np
import itertools
import os

In [9]:
fingers = ['1', '2', '3', '4', '5']

# Calculating valid sequences with rules

In [10]:
presses_per_finger= {
    1:1,
    2:2,
    3:1,
    4:2,
    5:1,
}

In [11]:
def check_illegal_seq(seq):
    #check runs of 3
    for pos in range(len(seq)- 2):
        if (int(seq[pos]) == int(seq[pos + 1]) + 1 and int(seq[pos+1]) == int(seq[pos+2]) + 1):
            return True
        if (int(seq[pos]) == int(seq[pos + 1]) - 1 and int(seq[pos+1]) == int(seq[pos+2]) - 1):
            return True
    
    #check rep press
    
    for pos in range(len(seq) - 1):
        if(int(seq[pos]) == int(seq[pos+1])):
            return True
        

    for finger in fingers:
        if seq.count(finger) == 0:
            return True
        

    # # match presses per finger 
    # for finger in fingers:
    #     if seq.count(finger) != presses_per_finger[int(finger)]:
    #         return True
        
    return False

In [12]:
check_illegal_seq('1243542')

False

In [13]:
valid_sequences = []

# generate all possible sequences of 7 digits from 1 to 5
for seq in itertools.product(range(1,6), repeat=7):
    seq = ''.join(map(str, seq))
    if not check_illegal_seq(seq):
        valid_sequences.append(seq)

In [14]:
len(valid_sequences)

5218

# Generating Target Files

In [15]:
# # windowSizes = ['1', '2', '3', '7']
# iti = 3000 
# # execTime = 10000 # msecs for each trial maximum
# # precueTime_interval = [600, 1000] # msecs for planning before movement 
# hand = 2 #left or right hand
# num_trained_seq = 1

# total_sub_num = 20
# num_sessions = 3
# num_blocks_per_session = 5

# num_baseline_blocks = 2
# num_trials_per_block = 20


In [16]:
# windowSizes = ['1', '2', '3', '7']
iti = 3000 
# execTime = 10000 # msecs for each trial maximum
# precueTime_interval = [600, 1000] # msecs for planning before movement 
hand = 2 #left or right hand
num_trained_seq = 1

total_sub_num = 20
num_sessions = 3
num_blocks_per_session = 4

num_baseline_blocks = 1
num_trials_per_block = 40


In [17]:
def determine_trained_seq_for_subj(subj_num):
    np.random.seed(subj_num)
    # while True:
    #     selected_sequences = np.random.choice(valid_sequences, num_trained_seq, replace=False)
    #     if sum([selected_sequences[0][i] == selected_sequences[1][i] for i in range(7)]) <= 2 and selected_sequences[0][0] != selected_sequences[1][0]:
    #         return selected_sequences

    selected_sequences = np.random.choice(valid_sequences, num_trained_seq, replace=False)
    return selected_sequences


In [18]:
for subnum in range(total_sub_num):
    print(determine_trained_seq_for_subj(subnum))

['2423251']
['1214532']
['3254124']
['1324353']
['2125341']
['2415313']
['5312141']
['1325451']
['4524213']
['4351325']
['3512434']
['1243545']
['3241425']
['2515342']
['5241535']
['2353145']
['4532351']
['5243135']
['3245341']
['2142523']


In [19]:
# trained_symbols = ['#', '$']
# random_symbol = '%'

In [20]:
### target file headers: [subNum, hand, isTrain, seq, iti]

for sub in range(total_sub_num):
    
    subNum = sub + 1
    hand = 2
    trained_seqs = determine_trained_seq_for_subj(subNum)
    bn = 1


    #### practice baseline block on the first day 
    for blocknum in range(num_baseline_blocks):
        block = pd.DataFrame(columns = ["subNum", "hand", "isTrain", "seq", "iti"])
        for _ in range(num_trials_per_block):
            while True:
                seq = np.random.choice(valid_sequences)
                if seq not in trained_seqs:
                    break
            trial = pd.DataFrame([[subNum, hand, 0, seq, iti]], columns = block.columns)
            block = pd.concat([block, trial], ignore_index=True)
        block = block.sample(frac=1).reset_index(drop = True)
        block.to_csv(f'SLR1_subj{subNum}_session1_block{bn}.tgt', index=False, sep='\t')
        bn += 1


    #### seq blocks 
    for session in range(num_sessions):
        for blocknum in range(num_blocks_per_session):
            block = pd.DataFrame(columns = ["subNum", "hand", "isTrain", "seq", "iti"])
            for _ in range(num_trials_per_block):
                trial = pd.DataFrame([[subNum, hand, 1, trained_seqs[0], iti]], columns = block.columns)
                block = pd.concat([block, trial], ignore_index=True)

            block = block.sample(frac=1).reset_index(drop = True)
            block.to_csv(f'SLR1_subj{subNum}_session{session+1}_block{bn}.tgt', index=False, sep='\t')
            bn += 1
    